# 유튜브 추천 알고리즘 설계


고객 데이터 생성

In [39]:
import pandas as pd

# 특정 고객의 데이터 생성
customer_data = {
    'CustomerID': [1],
    'Age': [65],
    'Gender': ['남자'],
    'Location': ['서울'],
    'Occupation': ['은퇴'],
    'Financial_Literacy_Level': ['보통'],
    'Investment_Horizon': ['장기투자'],
    'Risk_Tolerance': ['보수적'],
    'Preferred_Investment_Types': ['주식, 채권'],
    'Investment_Amount': [1000000],  # 투자 금액 (단위: 원)
    'Past_Investment_Experience': ['보통'],
    'Preferred_Investment_Topics': ['2차전지 관련주', '여행 관련주'],
    'Savings_Account': [0.65],
    'Bonds': [0.15],
    'Stocks': [0.2]
}

# 고객 특징 설명 글 생성
def generate_customer_description(customer_data):
    description = (
        f"이 고객은 {customer_data['Location'][0]}에 거주하는 {customer_data['Age'][0]}세의 {customer_data['Gender'][0]}입니다. "
        f"현재 직업은 {customer_data['Occupation'][0]}이며, 금융 이해도는 {customer_data['Financial_Literacy_Level'][0]} 수준입니다. "
        f"투자 기간은 {customer_data['Investment_Horizon'][0]}으로, 위험 감수 성향은 {customer_data['Risk_Tolerance'][0]}입니다. "
        f"주로 선호하는 투자 유형은 {customer_data['Preferred_Investment_Types'][0]}이며, 현재 투자 가능한 금액은 {customer_data['Investment_Amount'][0]}만원입니다. "
        f"과거 투자 경험은 {customer_data['Past_Investment_Experience'][0]} 수준이며, 선호하는 투자 주제는 {customer_data['Preferred_Investment_Topics'][0]}입니다. "
        f"현재 자산 구성 비율은 저축 계좌에 {customer_data['Savings_Account'][0]*100}%, 채권에 {customer_data['Bonds'][0]*100}%, 주식에 {customer_data['Stocks'][0]*100}%로 분포되어 있습니다."
    )
    return description

# 고객 특징 설명 글 출력
customer_feature_text = generate_customer_description(customer_data)
customer_feature_text

'이 고객은 서울에 거주하는 65세의 남자입니다. 현재 직업은 은퇴이며, 금융 이해도는 보통 수준입니다. 투자 기간은 장기투자으로, 위험 감수 성향은 보수적입니다. 주로 선호하는 투자 유형은 주식, 채권이며, 현재 투자 가능한 금액은 1000000만원입니다. 과거 투자 경험은 보통 수준이며, 선호하는 투자 주제는 2차전지 관련주입니다. 현재 자산 구성 비율은 저축 계좌에 65.0%, 채권에 15.0%, 주식에 20.0%로 분포되어 있습니다.'

유튜브 데이터 정제

In [36]:
import requests
import json

class CompletionExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def execute(self, completion_request):
        headers = {
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id,
            'Content-Type': 'application/json; charset=utf-8',
            'Accept': 'text/event-stream'
        }

        full_response = ""
        with requests.post(self._host + '/testapp/v1/chat-completions/HCX-003',
                           headers=headers, json=completion_request, stream=True) as r:
            for line in r.iter_lines():
                if line:
                    decoded_line = line.decode("utf-8")
                    if decoded_line.startswith("data:"):
                        json_data = decoded_line[5:].strip()  # 'data:' 이후의 내용 추출 및 공백 제거
                        try:
                            response_json = json.loads(json_data)
                            if "message" in response_json and "content" in response_json["message"] and response_json["outputLength"] != 1:
                                full_response += response_json["message"]["content"]
                        except json.JSONDecodeError:
                            print("Error: Failed to decode JSON")
        self.full_response = full_response
        # 최종 응답 출력

In [84]:
youtube = pd.read_csv('youtube_data_1.csv')[['title','content','video_url']]
youtube

,title,content,video_url
0,"[슬기로운 투자 생각] 중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨...",【이인구의 슬기로운 투자 생각】 ep4. 중국투자전략\n출연: 이인구 이사(미래에셋...,https://www.youtube.com/watch?v=JrHgv0R8RIQ&li...
1,"[슬기로운 투자 생각] 중국 고배당주 ETF, 투자 열풍 이유는? #Hangseng...",【이인구의 슬기로운 투자 생각】\nep5. 중국 Go! 배당 주!\n\n00:00 ...,https://www.youtube.com/watch?v=u1j5cB9Dqpc&li...
2,"반등 랠리 기대주 찾기 ""주가하락, 나는 진짜 억울해"" #CATL | 슬기로운 투자 생각","【이인구의 슬기로운 투자 생각】\nep.6 반등 랠리 기대주 찾기 ""주가하락, 나는...",https://www.youtube.com/watch?v=Ms6p1HznC_U&li...
3,"🌝중국 주도주 반등 랠리, 제대로 시동겁니다. 🚌 놓치기 전에 타세요~ [슬기로운 ...","【이인구의 슬기로운 투자 생각】\nep.6 반등 랠리 기대주 찾기 ""주가하락, 나는...",https://www.youtube.com/watch?v=7Yoej1DUR_0&li...
4,[유토피아] AI는 Apple iphone의 줄임말? | 2024.5.17(금),#apple #ai #openai\n\n✨ 쏟아지는 AI 관련 홍수 속에서 꼭 알아...,https://www.youtube.com/watch?v=GDHMzjCLSjk&li...
...,...,...,...
388,[오늘의마켓프리뷰] 연준 베이지북 공개 및 테슬라 실적 발표 대기ㅣ2023. 4. 19,"[연준 베이지북 공개 및 테슬라 실적 발표 대기]\n-한국증시, 보합권 등락 반복,...",https://www.youtube.com/watch?v=9V7C0ri6bB8&li...
389,"[오늘의마켓프리뷰] 한국증시, 지수 약세에도 2차전지 상승세 지속ㅣ2023.4.18",[주식시장 리뷰와 미 증시 전망]\nhttps://securities.miraeas...,https://www.youtube.com/watch?v=4BFmmjbyThw&li...
390,"[오늘의마켓프리뷰] 코스닥, 2차전지 강세와 개인 매수에 상승ㅣ2023.4.17","코스닥, 2차전지 강세와 개인 매수에 상승\n\n[▼디지털리서치보고서 보기▼]\n\...",https://www.youtube.com/watch?v=wW4Zh6xNcP0&li...
391,"[오늘의마켓프리뷰] 코스닥, 900선 돌파ㅣ2023.4.14","코스피, 6거래일 연속 상승 · 코스닥, 900선 돌파\n\n[▼디지털리서치보고서 ...",https://www.youtube.com/watch?v=phkNZSSX6UQ&li...


In [82]:
completion_executor = CompletionExecutor(
    host='https://clovastudio.stream.ntruss.com',
    api_key='NTA0MjU2MWZlZTcxNDJiY63IHRsuPbkmleZBGwzFCDyaHMUZtTU7D+D0PfCHfrjO',
    api_key_primary_val='826b9hfph6PTs9wV1QdkLh429u9T8PyraoBuR1FO',
    request_id='cb8206d2-5d4b-42d1-8ef1-af0486ab61cb'
)

summary = []

for i in range(30):
    preset_text = [
        {"role":"system","content":"<명령문>\n주어지는 title과 content는 유튜브 영상의 제목과 정보입니다. 이를 바탕으로 유튜브 영상의 주요 포인트를 요약해주세요. 특히 content에서 영상의 내용과 관련있는 정보만 따로 추출해서 가져와야 합니다. 이 요약은 다음의 기준을 따릅니다:\n- 제목이 핵심입니다. 제목을 바탕으로 내용을 유추해보세요.\n- 무조건 줄글로 전달. 개행문자 사용하지 않기.\n- 영상의 핵심 주제를 간결하게 전달\n- 중요한 세부 사항과 통계를 포함\n- 시청자에게 유용한 정보 제공\n- 200자 내외로 작성."},
        {"role": "user", "content": f"다음 유튜브의 정보를 파악하고 200 ~ 300자 사이로 요약하세요. 요약문은 무조건 줄글로 작성해야 합니다.  \n\n# title : {youtube.iloc[i].title}\n# content : {youtube.iloc[i].content}"}
    ]

    request_data = {
        'messages': preset_text,
        'topP': 0.8,
        'topK': 0,
        'maxTokens': 512,
        'temperature': 0.1,
        'repeatPenalty': 1.2,
        'stopBefore': [],
        'includeAiFilters': True,
        'seed': 0
    }
    completion_executor.execute(request_data)
    summary.append(completion_executor.full_response)
    print(completion_executor.full_response)

해당 영상에서는 중국의 매크로 상황을 점검하고, 해외 무역 비중의 변화와 주요 가전 업체들의 주가를 살펴봄으로써 중국 투자 전략에 대한 인사이트를 제공한다. 중국은 부동산 시장이 침체되어 있지만, 가전 업체들의 주가는 상승하고 있으며, 이는 중국의 해외 무역 비중이 변화하고 있기 때문이다. 이러한 변화를 바탕으로 중국 투자 전략을 어떻게 수립해야 하는지에 대한 팁을 제공한다.
해당 영상은 중국 고배당주 ETF 투자 열풍에 대한 내용을 다루고 있다. 중국 배당주 투자 열풍의 원인은 투자 대안의 부재와 높은 배당 수익률에 있다. 중국 정부의 밸류업 정책인 '신국9조'도 배당주 투자 열풍에 영향을 미치고 있다. 홍콩 배당주는 세제 혜택이 있으며, 배당+00은 더욱 중요해지고 있다. 중국과 홍콩 증시에서 고배당 투자가 새로운 투자 물결로 떠오르고 있다는 내용을 담고 있다.
해당 영상은 미래에셋증권의 이인구 이사가 출연하여, 글로벌 1위 배터리 업체인 CATL을 중심으로 한중 주요 배터리 제조업체의 수익성을 비교하고, CATL의 주가 및 PER 밴드를 분석하며 반등 랠리 기대주를 찾는 내용을 다루고 있다. 최근 주가 하락이 억울할 정도로 CATL의 성장성과 수익성이 뛰어나다는 점을 강조하며, 중국 주도주의 반등 랠리를 기대해볼 만하다는 의견을 제시한다.
해당 영상은 중국 주도주 중 하나인 글로벌 1위 배터리 업체 CATL을 다루고 있다. 한중 주요 배터리 제조업체 수익성 비교를 통해 CATL의 수익성이 높다는 것을 보여주며, CATL의 주가 및 PER 밴드를 통해 현재 주가가 매력적인 구간에 있다는 것을 알 수 있다.
해당 영상은 AI 관련 정보를 제공하는 영상으로, Open AI의 Spring Update와 GPT-4o의 성능에 대해 다루고 있다. 또한 AI Peer Table을 통해 다양한 AI 기술들의 성능을 비교해볼 수 있다. 미래에셋 스마트머니 채널에서는 해당 영상 외에도 다양한 AI 관련 리포트를 제공하고 있으니, 관심 있는 사람들은 해당 채널을 구독하여 정

유사도 측정

In [87]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Sentence Transformer 모델 (Most Likes 기준)
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# 고객 정보 벡터(문장 벡터)
sentence1_embedding = model.encode(customer_feature_text)

# 정제된 유튜브 content의 벡터(문장 벡터)로 결합된 행렬
embeddings = model.encode(summary)

# 한 문장 벡터를 2차원 배열로 변환
sentence1_embedding_2d = sentence1_embedding.reshape(1, -1)

# 코사인 유사도 계산
similarity = cosine_similarity(sentence1_embedding_2d, embeddings)
print(f"Cosine Similarity:\n{similarity}")


Cosine Similarity:
[[ 0.7999246   0.74044013  0.57176924  0.48870972  0.5961106   0.6889205
   0.796149    0.5645595   0.6956878  -0.01603384 -0.01603384 -0.01603384
   0.6227542   0.49617457  0.59673584  0.54402506  0.529436    0.6430816
   0.6394545   0.6297972   0.56761074  0.58187485  0.52939194  0.46657142
   0.6188198   0.5677366   0.45612928  0.7106632   0.640141    0.48531094]]


In [89]:
df = pd.DataFrame(columns = ['title', 'content', 'similarity', 'url'])
df['title'] = youtube.title[:30]
df['content'] = summary
df['similarity'] = similarity[0]
df['url'] = youtube.video_url[:30]
df.sort_values(by = 'similarity', ascending = False)

,title,content,similarity,url
0,"[슬기로운 투자 생각] 중국 부동산은 침체인데, 가전 업체들 주가는 UP... 무슨...","해당 영상에서는 중국의 매크로 상황을 점검하고, 해외 무역 비중의 변화와 주요 가전...",0.799925,https://www.youtube.com/watch?v=JrHgv0R8RIQ&li...
6,[유토피아] 유튜브로 학습하고 있었던 Open AI? | 2024.04.19(금),"해당 영상은 테슬라의 주가 하락과 인력 감축 소식을 전하며, AI와 자율주행 기술에...",0.796149,https://www.youtube.com/watch?v=g1WIY2Wvv_U&li...
1,"[슬기로운 투자 생각] 중국 고배당주 ETF, 투자 열풍 이유는? #Hangseng...",해당 영상은 중국 고배당주 ETF 투자 열풍에 대한 내용을 다루고 있다. 중국 배당...,0.740440,https://www.youtube.com/watch?v=u1j5cB9Dqpc&li...
27,[Charting Disruption 2023] Web 3.0 and The evo...,"이 영상은 글로벌엑스의 '차트링 디스럽션' 시리즈 중 하나로, 웹 3.0과 혁신 기...",0.710663,https://www.youtube.com/watch?v=BRRLDAXvVKg&li...
8,[GTC2024] Ep04. 메타버스 말고 이젠 디지털 트윈 | 2024.04.26(금),엔비디아가 개최한 세계 최대 인공지능(AI) 콘퍼런스인 'GTC 2024'에 다녀온...,0.695688,https://www.youtube.com/watch?v=dW0-nkdK9IU&li...
5,[유토피아] 중국에서 먼저 완전자율주행 출시하는 테슬라? | 2024.5.3(금),"해당 영상은 AI 관련 정보를 다루고 있으며, 마크 저커버그의 Llama3와 중화권...",0.688920,https://www.youtube.com/watch?v=th-bayS5DkI&li...
17,[ETF 로드맵🚩:기초편] Ep.06 뭐살지 몰라 다 준비했어 '채권형 ETF' |...,제목 '[ETF 로드맵🚩:기초편] Ep.06 뭐살지 몰라 다 준비했어 '채권형 ET...,0.643082,https://www.youtube.com/watch?v=Gi6kzi6ujoQ&li...
28,[Charting Disruption 2023] Forecast of next-ge...,이 영상은 미국의 노후화된 인프라와 차세대 인프라 투자에 대한 내용을 다루고 있다....,0.640141,https://www.youtube.com/watch?v=8_okHJxnm28&li...
18,[ETF 로드맵🚩:기초편] Ep.05 트렌드를 따라가는 테마형 ETF | 2024....,이 영상은 테마형 ETF에 대한 기초적인 내용을 다루고 있다. 테마형 ETF는 특정...,0.639454,https://www.youtube.com/watch?v=EJl4ISS0JB4&li...
19,[ETF 로드맵🚩:기초편] Ep.04 투자자라면 관심 많은 주제 '배당·인컴' | ...,제목 '[ETF 로드맵🚩:기초편] Ep.04 투자자라면 관심 많은 주제 '배당·인컴...,0.629797,https://www.youtube.com/watch?v=O5_0ZQEVuQY&li...


문자열 가중치 by GPT4

TF-IDF Weighted Cosine Similarity: 0.4335627257823944


가중치 적용해보자